# BERT finetuning on Yelp NYC data

Here, we will run the same experiment that we did on the 10000 Yelp samples, except over the entire dataset. 


In [1]:
!pip install bert-tensorflow

In [2]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0403 15:38:11.017744 139766446380928 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'yelp_finetuning_yelpzip'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'lucas0' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://lucas0/yelp_finetuning_yelpzip *****


In [0]:
from tensorflow import keras
import os
import re

def download_and_load_dataset(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="yelpZIP.txt", 
      origin="https://storage.googleapis.com/lucas0/yelpZIP.txt", 
      extract=False)
  dfile = open(dataset).readlines()
  return dfile

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

reviews = download_and_load_dataset()

In [5]:
data = {}
data['review'] = []
data['deceptive'] = []

for x in reviews:
  x = eval(x)
  data['review'].append(x[0])
  data['deceptive'].append(1 if x[1] else 0)
  
dataDict = pd.DataFrame.from_dict(data)

train, test = train_test_split(dataDict, shuffle=True, test_size=0.2)

print(len(train), len(test))

DATA_COLUMN = 'review'
LABEL_COLUMN = 'deceptive'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

128746 32187


In [6]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0403 15:38:23.654142 139766446380928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0403 15:38:26.453789 139766446380928 saver.py:1483] Saver not created because there are no variables in the graph to restore


Due to memory, we must set the max_seq_length to be 128, as any experiments with a higher figure used up all the GPU memory (12GB).

In [7]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 128746


I0403 15:38:27.703559 139766446380928 run_classifier.py:774] Writing example 0 of 128746


INFO:tensorflow:*** Example ***


I0403 15:38:27.715644 139766446380928 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0403 15:43:48.757504 139766446380928 run_classifier.py:774] Writing example 10000 of 32187


INFO:tensorflow:Writing example 20000 of 32187


I0403 15:44:11.435366 139766446380928 run_classifier.py:774] Writing example 20000 of 32187


INFO:tensorflow:Writing example 30000 of 32187


I0403 15:44:34.449536 139766446380928 run_classifier.py:774] Writing example 30000 of 32187


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size

BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0

# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.

WARMUP_PROPORTION = 0.1

# Model configs

SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [11]:
# Compute # train and warmup steps from batch size

num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps, num_warmup_steps)

12069 1206


In [0]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [13]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'gs://lucas0/yelp_finetuning_yelpzip', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d70671ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0403 15:44:39.362046 139766446380928 estimator.py:201] Using config: {'_model_dir': 'gs://lucas0/yelp_finetuning_yelpzip', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d70671ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.

train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

And now we begin training as normal. This will take ~6 hours.

In [15]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0403 15:45:56.555318 139766446380928 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0403 15:46:01.918440 139766446380928 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 12069 into gs://lucas0/yelp_finetuning_yelpzip/model.ckpt.


I0403 22:02:33.830096 139766446380928 basic_session_run_hooks.py:594] Saving checkpoints for 12069 into gs://lucas0/yelp_finetuning_yelpzip/model.ckpt.


INFO:tensorflow:Loss for final step: 0.53061444.


I0403 22:03:25.741516 139766446380928 estimator.py:359] Loss for final step: 0.53061444.


Training took time  6:18:46.263506


Now, let's evaluate..

In [0]:
test_input_fn = bert.run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [17]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0403 22:03:45.932313 139766446380928 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0403 22:03:51.182681 139766446380928 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 12069: auc = 0.6709257, eval_accuracy = 0.67092305, f1_score = 0.6726822, false_negatives = 5202.0, false_positives = 5390.0, global_step = 12069, loss = 0.81445044, precision = 0.66879684, recall = 0.6766132, true_negatives = 10711.0, true_positives = 10884.0


I0403 22:14:32.933892 139766446380928 estimator.py:1979] Saving dict for global step 12069: auc = 0.6709257, eval_accuracy = 0.67092305, f1_score = 0.6726822, false_negatives = 5202.0, false_positives = 5390.0, global_step = 12069, loss = 0.81445044, precision = 0.66879684, recall = 0.6766132, true_negatives = 10711.0, true_positives = 10884.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12069: gs://lucas0/yelp_finetuning_yelpzip/model.ckpt-12069


I0403 22:14:41.568455 139766446380928 estimator.py:2039] Saving 'checkpoint_path' summary for global step 12069: gs://lucas0/yelp_finetuning_yelpzip/model.ckpt-12069


{'auc': 0.6709257,
 'eval_accuracy': 0.67092305,
 'f1_score': 0.6726822,
 'false_negatives': 5202.0,
 'false_positives': 5390.0,
 'global_step': 12069,
 'loss': 0.81445044,
 'precision': 0.66879684,
 'recall': 0.6766132,
 'true_negatives': 10711.0,
 'true_positives': 10884.0}

Pretty disappointing result to be frank.

In [0]:
def getPrediction(in_sentences):
  labels = ["Deceptive", "Genuine"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "This hotel was truly terrible. Will never be returning.",
  "Rude staff, filthy room, crappy service. Won't be coming here again!",
  "I loved everything about my stay. The Hilton was a fantastic hotel and I will be returning.",
  "Absolutely fantastic! Great food, service and amenities. Looked after for the full duration."
]

In [20]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


I0403 22:14:43.586855 139766446380928 run_classifier.py:774] Writing example 0 of 4


INFO:tensorflow:*** Example ***


I0403 22:14:43.593309 139766446380928 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0403 22:14:50.942157 139766446380928 saver.py:1270] Restoring parameters from gs://lucas0/yelp_finetuning_yelpzip/model.ckpt-12069


INFO:tensorflow:Running local_init_op.


I0403 22:15:51.136852 139766446380928 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0403 22:15:51.303231 139766446380928 session_manager.py:493] Done running local_init_op.


In [21]:
predictions

[('This hotel was truly terrible. Will never be returning.',
  array([-1.4172978 , -0.27755755], dtype=float32),
  'Genuine'),
 ("Rude staff, filthy room, crappy service. Won't be coming here again!",
  array([-1.8109719 , -0.17852294], dtype=float32),
  'Genuine'),
 ('I loved everything about my stay. The Hilton was a fantastic hotel and I will be returning.',
  array([-1.5591199 , -0.23612879], dtype=float32),
  'Genuine'),
 ('Absolutely fantastic! Great food, service and amenities. Looked after for the full duration.',
  array([-1.9700981 , -0.15017565], dtype=float32),
  'Genuine')]